# Song Lyrics Generator
## Due Tuesday, May 17 at 8 AM

In this lab, you will scrape the web to get lyrics from your favorite artist. Then, you will train a Markov Chain model on these lyrics. Finally, you will use your Markov chain to generate new (random) lyrics.

## Question 1: Web Scraping (40 points)

Find a website that has lyrics for all the songs by your favorite artists. Then scrape the lyrics into a Python list called `lyrics`, where each element of the list represents the lyrics of one song.

Tips:
- Find a webpage that has links to all the songs, like [this one](http://www.azlyrics.com/n/nirvana.html). [NOTE: It seems like azlyrics.com does not allow you to scrape their webpages, so you'll have to find another source.] Then, you can write code to visit all the links and scrape each page one by one.
- Make sure you use time.sleep(0.1) to stagger your requests so that you do not get banned by the website for making too many requests.



In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import bs4

lyrics = []

# <table class = ..>
    #<tbody>
        #for all <tr> tag
            # array of <td>[1] //second td tag in list
                    
                #access a tag
                    #hyper link = arr of <td>[1]['href'] // to get the hyperlink of song lyrics
                    #go to hyperlink and parse the html
                
                    #<sd-lyricbody
                        #<div class = "...">
                            #<div id = "lyrics-body-text"
                                #find all p-class ="verse"
                                    #for each verse p tag
                                        #add pverse.string to temp variable to hold string of lyrics for single song
                                        #append string of lyrics to list of lyrics
                        

page = requests.get("http://www.metrolyrics.com/green-day-alpage-1.html")
time.sleep(0.1)
source = BeautifulSoup(page.text, "html.parser")

canStillRun = True

while(canStillRun == True):

    #print(source.find("a", class_= "button next disabled") == None)

    table_class = source.find("table", class_= "songs-table compact")
    #print( table_class != None)

    t_body = table_class.find("tbody")
    tr_s = t_body.find_all("tr") #returns a list of trs that contain link to other page (links to lyrics of songs)

    for tr in tr_s: #for each song.. on page x...
        
        list_of_tds = tr.find_all("td")
        
        
        atag_under_tds_list = list_of_tds[1].find('a')

        href_hyperlink = atag_under_tds_list['href']

        #go to link to lyrics
        lyric_page = requests.get(href_hyperlink)
        time.sleep(0.1)
        lyric_source = BeautifulSoup(lyric_page.text, "html.parser")
       
        div_id_lyrics_body = lyric_source.find('div', class_ = "js-lyric-text")

        tmp_lyrics_singlesong = ""

        verses_list = div_id_lyrics_body.find_all('p')

        for verse_tag in verses_list:
    
            verse_filtered = str(verse_tag).replace("<p class=\"verse\">", "" ).replace('<br>', "").replace("</br>", "").replace("\n", " ").replace("</p>", "\n\n")
            tmp_lyrics_singlesong += verse_filtered
    
        lyrics.append(tmp_lyrics_singlesong)

    #After done adding lyrics of each song to main list, advance to next page
    
    #if next button is disabled, break out of loop
    
    if(source.find("a", class_= "button next disabled") == None):
        
        a_tag_toNextPage = source.find("a", class_= "button next")
        href_hyperlink = a_tag_toNextPage['href']
        
        page = requests.get(href_hyperlink)
        time.sleep(0.1)
        source = BeautifulSoup(page.text, "html.parser")
    else:
        
        canStillRun = False



    
    
    

In [ ]:
print(lyrics[0])

## Question 2: Training a Markov Chain (30 points)

Markov chains are mathematical systems that hop (a.k.a. "transition") randomly between various states. Please read [this visual explanation](http://setosa.io/ev/markov-chains/) for a high-level overview. The distinguishing feature of a Markov chain is that the next state only depends on which state the chain is in now; it doesn't depend on the past history of the chain.

We can use Markov chains to model human language. Each word is a "state", and the next word in a sentence only depends on the current word, not any words that came before. This model makes sense because if we know that the current word is "it", the next word is very likely to be "is", less likely to be "runs", and never going to be "pineapple". On the other hand, if the current word is "the", then the next word might be "pineapple", but it can't be "is". The current word tells us a lot about what the next word might be.

We will build a Markov chain model for the artist whose lyrics you scraped in Question 1. To do this, we have to go through the lyrics and learn the word transitions for that artist. We will store this information in a dict called `transitions`, which maps each word to a list of words that appear after it in the training data. So for example, one entry of this dict might be

```
transitions = {
    "it": ["is", "runs", "is", "is", "was", "is", "was"],
    ...
}
```

You should include a few additional states, besides words, in your Markov chain. You should have `"<START>"` and `"<END>"` states so that we can keep track of what words songs are likely to begin and end on.  So if the song starts on the word "it" and ends with the word "me.", you would have 
```
transitions = {
    "<START>": ["it", ...],
    "me.": ["<END>", ...],
    ...
}
```
You should also include a state called `"<N>"` to denote line breaks so that we know where lines begin and end.

In [ ]:
def rreplace(s, old, new, occurrence): #works on a string, not list; replaces last n occurences of old instance with new
    li = s.rsplit(old, occurrence)
    return new.join(li)

def train_markov_chain(lyrics):
    transitions = {"<START>": []}
    
    for lyric in lyrics:
        
        string1 = lyric.replace("\n\n", " <N> ")
        string1b = rreplace(string1, "<N>", "", 1)

        string2 = "<START> " + string1b + "<END>"
        string3 = string2.split(" ") #each song lyrics are broken up into a list of words.
        string3.remove("")


        index = 0 # for list of words

        for word in string3:

            #print(word)

            if word not in transitions:

                transitions[word] = []
                if index + 1 < len(string3):

                    transitions[word].append(string3[index+1])
            else:

                if index + 1 < len(string3):

                    transitions[word].append(string3[index+1])

            index += 1
    
    #print(string3)
    return transitions

transitions = {}


    
for lyric in lyrics:
        
    string1 = lyric.replace("\n\n", " <N> ")
    string1b = rreplace(string1, "<N>", "", 1)
    
    string2 = "<START> " + string1b + "<END>"
    string3 = string2.split(" ") #each song lyrics are broken up into a list of words.
    string3.remove("")
    
    
    index = 0 # for list of words
    
    for word in string3:
        
        #print(word)
        
        if word not in transitions:
            
            transitions[word] = []
            if index + 1 < len(string3):

                transitions[word].append(string3[index+1])
        else:
            
            if index + 1 < len(string3):

                transitions[word].append(string3[index+1])
            
        index += 1
    
    #print(string3)
transitions

 

In [ ]:
chain = train_markov_chain(lyrics)
print(chain["<START>"][:20])
print(chain["<N>"][:20])

## Question 3: Generating New Lyrics (20 points)

Finally, let's generate new lyrics using the Markov chain you constructed in Question 2. To do this, we'll begin at the `"<START>"` state and randomly sample a word from the list of first words. Then, we'll randomly sample each next word from the list of words that appeared after the current word in the training data. We will continue this until we reach the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

You may find the `choice()` function in the `random` package helpful for this question.

In [ ]:
import random

def generate_new_lyrics(chain):
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain["<START>"]))
    
    for word in chain.keys():
        if len(chain[word]) != 0:
            words.append(random.choice(chain[word]))
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [ ]:
print(generate_new_lyrics(chain))